In [1]:
!pip install -q "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import pandas as pd
from datasets import Dataset

dataset_df = pd.read_csv('/content/category_find.csv')
df = dataset_df.sample(frac=1).reset_index(drop=True)
df.head()

,Sentence,Categories,Response,Rationale for Relevant Categories
0,The recipe calls for two cups of flour.,"Cooking, Measurement, Shopping, Nutrition",Cooking: Relevant | Measurement: Relevant | Sh...,"Cooking: Discussing a recipe, directly related..."
1,The sky was filled with hues of orange and pink.,"Transportation, Food, Psychology, Technology, ...",Transportation: Not Applicable | Food: Not App...,None of the categories relate to describing th...
2,The solar panel converts sunlight into energy.,"Renewable Energy, Technology, Nature, Physics",Renewable Energy: Relevant | Technology: Relev...,Renewable Energy: Converting sunlight to energ...
3,The software development process included vers...,"Technology, Software Development, Quality, Pro...",Technology: Relevant | Software Development: R...,Technology: The topic is software development ...
4,She wore her favorite bracelet.,"Science, Education, Politics, Healthcare, Travel",Science: Not Applicable | Education: Not Appli...,None of the categories directly relate to wear...


In [4]:
df['Final_Response'] = df['Response'] + "\n" + df['Rationale for Relevant Categories']

In [5]:
df.head()

,Sentence,Categories,Response,Rationale for Relevant Categories,Final_Response
0,The recipe calls for two cups of flour.,"Cooking, Measurement, Shopping, Nutrition",Cooking: Relevant | Measurement: Relevant | Sh...,"Cooking: Discussing a recipe, directly related...",Cooking: Relevant | Measurement: Relevant | Sh...
1,The sky was filled with hues of orange and pink.,"Transportation, Food, Psychology, Technology, ...",Transportation: Not Applicable | Food: Not App...,None of the categories relate to describing th...,Transportation: Not Applicable | Food: Not App...
2,The solar panel converts sunlight into energy.,"Renewable Energy, Technology, Nature, Physics",Renewable Energy: Relevant | Technology: Relev...,Renewable Energy: Converting sunlight to energ...,Renewable Energy: Relevant | Technology: Relev...
3,The software development process included vers...,"Technology, Software Development, Quality, Pro...",Technology: Relevant | Software Development: R...,Technology: The topic is software development ...,Technology: Relevant | Software Development: R...
4,She wore her favorite bracelet.,"Science, Education, Politics, Healthcare, Travel",Science: Not Applicable | Education: Not Appli...,None of the categories directly relate to wear...,Science: Not Applicable | Education: Not Appli...


In [6]:
INTRO_BLURB = "Below is an instruction that describes a task. Write response that appropriately completes the request."
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Sentence:"
CAT_KEY = "Categories"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"



PROMPT_WITH_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{input_key}
{input}

{cat_key}
{cat_input}

{response_key}
{response}

{end_key}""".format(
  intro=INTRO_BLURB,
  instruction_key=INSTRUCTION_KEY,
  instruction="{instruction}",
  input_key=INPUT_KEY,
  input="{input}",
  cat_key=CAT_KEY,
  cat_input="{cat_input}",
  response_key=RESPONSE_KEY,
  response="{response}",
  end_key=END_KEY
)

In [7]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['Sentence', 'Categories', 'Response', 'Rationale for Relevant Categories', 'Final_Response'],
    num_rows: 16
})

In [9]:
def apply_prompt_template(examples):
  instruction = "Find the suitable categories and the rational things for given sentence"
  context = examples.get("Sentence")
  cat_res = examples.get("Categories")
  response = examples["Final_Response"]

  if context:
    full_prompt = PROMPT_WITH_INPUT_FORMAT.format(instruction=instruction, response=response, input=context, cat_input=cat_res)

  return { "text": full_prompt }

dataset = dataset.map(apply_prompt_template)


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [10]:
print(dataset["text"][0])

Below is an instruction that describes a task. Write response that appropriately completes the request.

### Instruction:
Find the suitable categories and the rational things for given sentence

Sentence:
The recipe calls for two cups of flour.

Categories
Cooking, Measurement, Shopping, Nutrition

### Response:
Cooking: Relevant | Measurement: Relevant | Shopping: Not Applicable | Nutrition: Not Applicable
Cooking: Discussing a recipe, directly related to cooking | Measurement: Specifies "two cups of flour," a unit of measurement

### End


In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=bnb_config,
    device_map="cuda:0",
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'k_proj', 'v_proj'] # Choose all linear layers from the model
)

In [13]:
from transformers import TrainingArguments

output_dir = "results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 5
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 26
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    ddp_find_unused_parameters=False,
)


In [14]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [15]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [16]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.213800
2,2.074200
3,1.890000
4,1.674900
5,1.480300
6,1.274300
7,1.096800
8,0.923200
9,0.766100
10,0.645700


TrainOutput(global_step=26, training_loss=0.7089922079959741, metrics={'train_runtime': 130.4495, 'train_samples_per_second': 3.189, 'train_steps_per_second': 0.199, 'total_flos': 1605441147568128.0, 'train_loss': 0.7089922079959741, 'epoch': 26.0})

In [17]:
from transformers import pipeline

In [18]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [19]:
prompt = f"""
Below is an instruction that describes a task. Write response that appropriately completes the request.

### Instruction:
Find the suitable categories and the rational things for given sentence

Sentence:
I have done a deeplearning research work last week.

Categories
AI, machine learning, cooking, driving, computer

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")



Below is an instruction that describes a task. Write response that appropriately completes the request.

### Instruction:
Find the suitable categories and the rational things for given sentence

Sentence:
I have done a deeplearning research work last week.

Categories
AI, machine learning, cooking, driving, computer

### Response:
AI: Relevant | Machine Learning: Relevant | Cooking: Not Applicable | Driving: Not Applicable | Computer: Relevant
AI is related to deeplearning, and computer science is the context of the sentence.


None
